In [1]:
import pandas as pd
import talib
import pynance as pn
import yfinance as yf
import pandas_ta as ta
from IPython.display import display
import os

In [2]:
import sys
sys.path.append('../scripts')

In [3]:
from quantitative import load_stock_data, add_technical_indicators, add_trading_signals, calculate_financial_metrics, backtest_signals, print_statistical_summary, plot_price_with_indicators



In [5]:
stocks = {
    'AAPL': '../data/yfinance_data/AAPL_historical_data.csv',
    'AMZN': '../data/yfinance_data/AMZN_historical_data.csv',
    'GOOG': '../data/yfinance_data/GOOG_historical_data.csv',
    'META': '../data/yfinance_data/META_historical_data.csv',
    'MSFT': '../data/yfinance_data/MSFT_historical_data.csv',
    'NVDA': '../data/yfinance_data/NVDA_historical_data.csv',
    'TSLA': '../data/yfinance_data/TSLA_historical_data.csv'
}
stock_dfs = {}

In [6]:
def apply_analysis_step(func, stock_dfs, func_name, columns=None, drop_na=False, **kwargs):
    print(f"\n{func_name}...")
    for ticker, df in stock_dfs.items():
        print(f"Processing {ticker}...")
        stock_dfs[ticker] = func(df, **kwargs)
        if columns:
            display_df = stock_dfs[ticker][columns]
            if drop_na:
                display_df = display_df.dropna()
            display(display_df.head())
    return stock_dfs

In [7]:
print("Loading and filtering data (post-2018 for relevance)...")
for ticker, filepath in stocks.items():
    print(f"Loading {ticker}...")
    df = load_stock_data(filepath=filepath)
    if df is not None:
        # Filter for recent data to reduce NaN and align with news
        df = df[df.index >= '2023-01-01']  # Adjusted to 2023 for recent trends
        if len(df) < 50:
            print(f"Warning: {ticker} has insufficient data ({len(df)} rows)")
            continue
        stock_dfs[ticker] = df
        display(df[['Close', 'Open', 'High', 'Low', 'Volume']].head())
    else:
        print(f"Failed to load data for {ticker}")

Loading and filtering data (post-2018 for relevance)...
Loading AAPL...


,Close,Open,High,Low,Volume
Date,,,,,
2023-01-03,125.070000,130.279999,130.899994,124.169998,112117500
2023-01-04,126.360001,126.889999,128.660004,125.080002,89113600
2023-01-05,125.019997,127.129997,127.769997,124.760002,80962700
2023-01-06,129.619995,126.010002,130.289993,124.889999,87754700
2023-01-09,130.149994,130.470001,133.410004,129.889999,70790800


Loading AMZN...


,Close,Open,High,Low,Volume
Date,,,,,
2023-01-03,85.820000,85.459999,86.959999,84.209999,76706000
2023-01-04,85.139999,86.550003,86.980003,83.360001,68885100
2023-01-05,83.120003,85.330002,85.419998,83.070000,67930800
2023-01-06,86.080002,83.029999,86.400002,81.430000,83303400
2023-01-09,87.360001,87.459999,89.480003,87.080002,65266100


Loading GOOG...


...

,Close,Open,High,Low,Volume
Date,,,,,
2023-01-03,239.580002,243.080002,245.750000,237.399994,25740000
2023-01-04,229.100006,232.279999,232.869995,225.960007,50623400
2023-01-05,222.309998,227.199997,227.550003,221.759995,39585600
2023-01-06,224.929993,223.000000,225.759995,219.350006,43613600
2023-01-09,227.119995,226.449997,231.240005,226.410004,27369800


Loading NVDA...


,Close,Open,High,Low,Volume
Date,,,,,
2023-01-03,14.315,14.851,14.996,14.096,401277000
2023-01-04,14.749,14.567,14.853,14.241,431324000
2023-01-05,14.265,14.491,14.564,14.148,389168000
2023-01-06,14.859,14.474,15.010,14.034,405044000
2023-01-09,15.628,15.284,16.056,15.141,504231000


Loading TSLA...


,Close,Open,High,Low,Volume
Date,,,,,
2023-01-03,108.099998,118.470001,118.800003,104.639999,231402800
2023-01-04,113.639999,109.110001,114.589996,107.519997,180389000
2023-01-05,110.339996,110.510002,111.750000,107.160004,157986300
2023-01-06,113.059998,103.000000,114.389999,101.809998,220911100
2023-01-09,119.769997,118.959999,123.519997,117.110001,190284000


In [8]:
stock_dfs = apply_analysis_step(
    add_technical_indicators,
    stock_dfs,
    "Adding technical indicators",
    columns=['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'BB_upper'],
    drop_na=True
)


Adding technical indicators...
Processing AAPL...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2023-03-15,152.990005,150.305501,144.7934,58.933103,2.013153,155.895566
2023-03-16,155.850006,150.331501,145.4090,63.251906,2.261574,156.018795
2023-03-17,155.000000,150.396001,145.9818,61.192187,2.362626,156.261560
2023-03-20,157.399994,150.638500,146.6294,64.688595,2.606326,157.199963
2023-03-21,159.279999,151.178500,147.2226,67.182832,2.917530,158.654440


Processing AMZN...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2023-03-15,96.199997,94.5550,95.9290,53.898922,-0.633126,99.235661
2023-03-16,100.040001,94.4990,96.2134,61.225283,-0.112257,98.879397
2023-03-17,98.949997,94.5390,96.4896,58.388752,0.210158,99.064239
2023-03-20,97.709999,94.5645,96.7814,55.252669,0.361450,99.154712
2023-03-21,100.610001,94.8660,97.0720,60.584619,0.707204,100.159015


Processing GOOG...


...

,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2023-03-15,265.440002,254.786500,249.566400,61.144715,2.215943,266.271816
2023-03-16,276.200012,255.130500,250.298801,68.199836,3.665669,268.580492
2023-03-17,279.429993,255.994500,251.305400,69.962962,5.017382,272.910608
2023-03-20,272.230011,256.703001,252.303801,61.744869,5.444881,275.033629
2023-03-21,273.779999,257.758501,253.280801,62.759026,5.841411,277.421343


Processing NVDA...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2023-03-15,24.228001,22.99065,20.45718,62.777335,1.001864,25.020888
2023-03-16,25.541000,23.12950,20.68170,68.349950,1.097727,25.439334
2023-03-17,25.725000,23.31565,20.90122,69.049245,1.175001,25.823597
2023-03-20,25.900000,23.54125,21.13392,69.734169,1.236114,26.125592
2023-03-21,26.198999,23.81845,21.36072,70.918326,1.293759,26.291930


Processing TSLA...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2023-03-15,180.449997,193.732000,170.3836,47.867344,1.630880,218.079173
2023-03-16,184.130005,192.226500,171.9042,50.366366,1.446184,214.987051
2023-03-17,180.130005,191.131001,173.2340,47.690362,0.965910,214.005579
2023-03-20,183.250000,189.878001,174.6922,49.925189,0.827508,211.565895
2023-03-21,197.580002,189.888501,176.3826,58.660935,1.852777,211.591092


In [9]:
print("\nAdding technical indicators...")
for ticker, df in stock_dfs.items():
    print(f"Processing {ticker}...")
    stock_dfs[ticker] = add_technical_indicators(df)
    display(stock_dfs[ticker][['Close', 'SMA_20', 'SMA_50', 'RSI', 'MACD', 'BB_upper']].head())




Adding technical indicators...
Processing AAPL...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
1980-12-12,0.128348,NaN,NaN,NaN,NaN,NaN
1980-12-15,0.121652,NaN,NaN,NaN,NaN,NaN
1980-12-16,0.112723,NaN,NaN,NaN,NaN,NaN
1980-12-17,0.115513,NaN,NaN,NaN,NaN,NaN
1980-12-18,0.118862,NaN,NaN,NaN,NaN,NaN


Processing AMZN...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
1997-05-15,0.097917,NaN,NaN,NaN,NaN,NaN
1997-05-16,0.086458,NaN,NaN,NaN,NaN,NaN
1997-05-19,0.085417,NaN,NaN,NaN,NaN,NaN
1997-05-20,0.081771,NaN,NaN,NaN,NaN,NaN
1997-05-21,0.071354,NaN,NaN,NaN,NaN,NaN


Processing GOOG...


...

,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
1986-03-13,0.097222,NaN,NaN,NaN,NaN,NaN
1986-03-14,0.100694,NaN,NaN,NaN,NaN,NaN
1986-03-17,0.102431,NaN,NaN,NaN,NaN,NaN
1986-03-18,0.099826,NaN,NaN,NaN,NaN,NaN
1986-03-19,0.098090,NaN,NaN,NaN,NaN,NaN


Processing NVDA...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
1999-01-22,0.041016,NaN,NaN,NaN,NaN,NaN
1999-01-25,0.045313,NaN,NaN,NaN,NaN,NaN
1999-01-26,0.041797,NaN,NaN,NaN,NaN,NaN
1999-01-27,0.041667,NaN,NaN,NaN,NaN,NaN
1999-01-28,0.041536,NaN,NaN,NaN,NaN,NaN


Processing TSLA...


,Close,SMA_20,SMA_50,RSI,MACD,BB_upper
Date,,,,,,
2010-06-29,1.592667,NaN,NaN,NaN,NaN,NaN
2010-06-30,1.588667,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.464000,NaN,NaN,NaN,NaN,NaN
2010-07-02,1.280000,NaN,NaN,NaN,NaN,NaN
2010-07-06,1.074000,NaN,NaN,NaN,NaN,NaN


In [9]:
stock_dfs = apply_analysis_step(
    add_trading_signals,
    stock_dfs,
    "Adding trading signals",
    columns=['Close', 'Signal']
)


Adding trading signals...
Processing AAPL...


,Close,Signal
Date,,
2023-01-03,125.070000,0
2023-01-04,126.360001,0
2023-01-05,125.019997,0
2023-01-06,129.619995,0
2023-01-09,130.149994,0


Processing AMZN...


,Close,Signal
Date,,
2023-01-03,85.820000,0
2023-01-04,85.139999,0
2023-01-05,83.120003,0
2023-01-06,86.080002,0
2023-01-09,87.360001,0


Processing GOOG...


...

,Close,Signal
Date,,
2023-01-03,239.580002,0
2023-01-04,229.100006,0
2023-01-05,222.309998,0
2023-01-06,224.929993,0
2023-01-09,227.119995,0


Processing NVDA...


,Close,Signal
Date,,
2023-01-03,14.315,0
2023-01-04,14.749,0
2023-01-05,14.265,0
2023-01-06,14.859,0
2023-01-09,15.628,0


Processing TSLA...


,Close,Signal
Date,,
2023-01-03,108.099998,0
2023-01-04,113.639999,0
2023-01-05,110.339996,0
2023-01-06,113.059998,0
2023-01-09,119.769997,0


In [10]:
stock_dfs = apply_analysis_step(
    calculate_financial_metrics,
    stock_dfs,
    "Calculating financial metrics",
    columns=['Close', 'Daily_Return', 'Volatility', 'Sharpe_Ratio']
)


Calculating financial metrics...
Processing AAPL...


,Close,Daily_Return,Volatility,Sharpe_Ratio
Date,,,,
2023-01-03,125.070000,NaN,NaN,NaN
2023-01-04,126.360001,0.010314,NaN,NaN
2023-01-05,125.019997,-0.010605,NaN,NaN
2023-01-06,129.619995,0.036794,NaN,NaN
2023-01-09,130.149994,0.004089,NaN,NaN


Processing AMZN...


,Close,Daily_Return,Volatility,Sharpe_Ratio
Date,,,,
2023-01-03,85.820000,NaN,NaN,NaN
2023-01-04,85.139999,-0.007924,NaN,NaN
2023-01-05,83.120003,-0.023726,NaN,NaN
2023-01-06,86.080002,0.035611,NaN,NaN
2023-01-09,87.360001,0.014870,NaN,NaN


Processing GOOG...


...

,Close,Daily_Return,Volatility,Sharpe_Ratio
Date,,,,
2023-01-03,239.580002,NaN,NaN,NaN
2023-01-04,229.100006,-0.043743,NaN,NaN
2023-01-05,222.309998,-0.029638,NaN,NaN
2023-01-06,224.929993,0.011785,NaN,NaN
2023-01-09,227.119995,0.009736,NaN,NaN


Processing NVDA...


,Close,Daily_Return,Volatility,Sharpe_Ratio
Date,,,,
2023-01-03,14.315,NaN,NaN,NaN
2023-01-04,14.749,0.030318,NaN,NaN
2023-01-05,14.265,-0.032816,NaN,NaN
2023-01-06,14.859,0.041640,NaN,NaN
2023-01-09,15.628,0.051753,NaN,NaN


Processing TSLA...


,Close,Daily_Return,Volatility,Sharpe_Ratio
Date,,,,
2023-01-03,108.099998,NaN,NaN,NaN
2023-01-04,113.639999,0.051249,NaN,NaN
2023-01-05,110.339996,-0.029039,NaN,NaN
2023-01-06,113.059998,0.024651,NaN,NaN
2023-01-09,119.769997,0.059349,NaN,NaN


In [11]:
print("\nStatistical summaries...")
for ticker, df in stock_dfs.items():
    print_statistical_summary(df, ticker)
    # Additional insight: Signal success rate
    if 'Signal' in df.columns:
        trades = df['Signal'].diff().abs().where(df['Signal'].diff().abs() > 0).dropna()
        profitable_trades = df.loc[trades.index, 'Daily_Return'].where(df['Signal'].shift(1) == 1).dropna()
        success_rate = (profitable_trades > 0).mean() if len(profitable_trades) > 0 else 0
        print(f"Signal Success Rate (Buy signals leading to positive returns): {success_rate:.2%}")


Statistical summaries...

Statistical Summary for AAPL
--------------------------------------------------
Price Statistics:


,Close,Daily_Return,Volatility,Sharpe_Ratio
count,395.000000,394.000000,374.000000,374.000000
mean,178.978684,0.001514,0.210240,1.664855
std,19.703762,0.013712,0.052023,3.715074
min,125.019997,-0.048020,0.120892,-9.175073
25%,169.340004,-0.006881,0.165655,-0.714766
50%,179.070007,0.001606,0.210948,1.959834
75%,189.974998,0.008725,0.245069,4.597809
max,234.820007,0.072649,0.340061,10.720557



Indicator Statistics:


,SMA_20,SMA_50,RSI,MACD,BB_upper,BB_lower
count,376.000000,346.000000,381.000000,362.000000,376.000000,376.000000
mean,179.042738,178.907181,56.577489,1.504155,188.012431,170.073045
std,16.623318,12.297647,13.818548,3.043059,18.262950,15.755033
min,135.778999,144.793400,22.018390,-4.418886,148.714720,122.843278
25%,171.471625,174.145400,44.589881,-1.364250,176.758158,163.951263
50%,179.644751,180.093501,59.388374,2.396415,189.992435,169.322098
75%,188.712624,186.756350,67.038532,3.497257,196.845777,180.253413
max,225.117001,210.858600,90.119479,8.948060,239.820553,214.234202



Correlation Matrix:


...


Indicator Statistics:


,SMA_20,SMA_50,RSI,MACD,BB_upper,BB_lower
count,376.000000,346.000000,381.000000,362.000000,376.000000,376.000000
mean,209.946718,211.389848,52.318151,1.372737,235.097462,184.795973
std,34.405740,30.426570,14.773692,8.549425,38.225811,35.266217
min,134.195999,170.383600,17.427926,-14.073484,174.868540,93.015838
25%,178.443125,181.992400,43.412639,-4.737581,201.435014,159.603776
50%,199.641751,205.416000,51.377923,1.008256,228.965956,175.465976
75%,241.973749,237.833400,59.940815,4.713828,271.777797,215.712767
max,273.505500,259.138400,89.912813,23.206692,298.187291,252.984557



Correlation Matrix:


,Close,SMA_20,SMA_50,RSI,MACD,Daily_Return
Close,1.000000,0.843319,0.646814,0.347381,0.515807,0.018747
SMA_20,0.843319,1.000000,0.857633,-0.119783,0.177073,-0.130765
SMA_50,0.646814,0.857633,1.000000,-0.272287,-0.209095,-0.107961
RSI,0.347381,-0.119783,-0.272287,1.000000,0.826040,0.369976
MACD,0.515807,0.177073,-0.209095,0.826040,1.000000,0.032416
Daily_Return,0.018747,-0.130765,-0.107961,0.369976,0.032416,1.000000



Trading Strategy Performance:
Cumulative Return: -56.23%
Number of Trades: 28
Signal Success Rate (Buy signals leading to positive returns): 92.31%


In [13]:
print("\nVisualizing indicators...")
plot_count = 0
max_plots = 10  # Per challenge requirement
for ticker, df in stock_dfs.items():
    if plot_count >= max_plots:
        print("Reached maximum plot limit (10). Skipping remaining plots.")
        break
    print(f"Plotting {ticker}...")
    #plot_price_with_indicators(df, ticker, save_path=f'{output_dir}/{ticker}_plots.png')
    plot_count += 1


Visualizing indicators...
Plotting AAPL...
Plotting AMZN...
Plotting GOOG...
Plotting META...
Plotting MSFT...
Plotting NVDA...
Plotting TSLA...


In [ ]:
print("\nComparative Analysis Across Stocks")
metrics = []
for ticker, df in stock_dfs.items():
    cum_return, num_trades = backtest_signals(df)
    metrics.append({
        'Ticker': ticker,
        'Mean Daily Return (%)': df['Daily_Return'].mean() * 252 * 100,
        'Volatility (%)': df['Volatility'].mean() * 100,
        'Sharpe Ratio': df['Sharpe_Ratio'].mean(),
        'Cumulative Strategy Return (%)': cum_return * 100,
        'Number of Trades': int(num_trades)
    })
comparison_df = pd.DataFrame(metrics)
display(comparison_df)